#  Some slightly hacky stuff to do around pycortex 

Easy (and hacky!!) way to import freesurfer ROIs into pycortex

BEFORE YOU DO THIS MAKE SURE TO BACK UP ANY KIND OF PYCORTEX STUFF YOU DON'T WANT OVERWRITTEN!!!

Why would you do this? Because pycortex is beautiful, but you are lazy... 

It is really nice to be able to toggle the ROIs on and off in pycortex viewer; but to do this you have to have them defined on the svg. To define them on the SVG, you need to draw them in inkscape. But maybe you already used freesurfer, and don't want to... whatever

This helps with that...

It also works with anything defined as a boolean array. 

Also for plotting flatmaps in a pycortex like way but with more flexibility with matplotlib could be useful


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
opj = os.path.join

In [ ]:
# FOR MY OWN TESTING OF THIS EXAMPLE FOLDER
fs_dir = '/data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/freesurfer'
ctx_dir = '/data1/projects/dumoulinlab/Lab_members/Marcus/projects/pilot1/derivatives/pycortex'
from pfa_scripts.load_saved_info import *
from dag_prf_utils.prfpy_functions import Prf1T1M
prfs = Prf1T1M(
    load_data_prf('sub-02', 'AS0', 'gauss')['AS0']['gauss'],
    model='gauss',)
polar_angle_data = prfs.pd_params['pol'].to_numpy()
eccentricity_data = prfs.pd_params['ecc'].to_numpy()
data_mask = prfs.return_vx_mask(th={'min-rsq':.1, 'max-ecc':5})
data_rsq = prfs.pd_params['rsq'].to_numpy()
data_alpha = data_rsq.copy()
sub='sub-02'

In [ ]:
# sub = 'sub-01'                                      # Subject number
# fs_dir = '~/home/project1/derivatives/freesurfer'   # e.g., /home/project1/derivatives/freesurfer
# ctx_dir = '~/home/project1/derivatives/pycortex' # e.g., /home/project1/derivatives/freesurfer/label



In [ ]:
from dag_prf_utils.pycortex import set_ctx_path,PyctxMaker

# Make sure pycortex is pointing to correct folder
# set_ctx_path('/path/to/pycortex/files')

pm = PyctxMaker(
    sub = sub,
    fs_dir = fs_dir, 
    ctx_path=ctx_dir,
    )


## quick flatmaps
Already got flatmaps? then skip this stage

Pycortex needs flatmaps. In the form of flat_*h.gii in the pycortex folder for that subject. If you are very lazy (like me) and don't want to go through the trouble of flattening - i offer a couple of DODGY ways  to flatten the surface. 

It involves selecting some of the vertices to focus on. Then applying some kind of flattening algorithm. I then save the stuff into the pycortex folder

In [ ]:
import inspect
from dag_prf_utils.utils import dag_load_roi
# Lets pick some vertices to focus on:
# I have drawn a bunch of ROIs in the freesurfer folder
# all called 'custom', so I'll load them
# But you can make this selection anyway you want
centre_bool = dag_load_roi(
    sub=sub,
    roi='v1',
    fs_dir=fs_dir,
    combine_matches=True,  
)

print(f'Number of vertices: {centre_bool.sum()}')
# Boolean array of vertices (in both hemispheres)

Make the flatmap 

Basically take the focussing on the coordinates, we can do some flattening. This can be primitively done using the freesurfer projection to a sphere (and just using latitude and longitude). Or you can use the "igl" package which does some clever as-rigid-as-possible transformations. IGL is better, but can fail randomly during the making - so you may have to rerun until you are happy

In [ ]:
print(inspect.getsource(pm.make_flat_map))

In [ ]:
if True: # DONT DO IT THE SECOND TIME
    pm.mesh_info['occ'] = None
    pm.make_flat_map(
        centre_bool=centre_bool,
        method='igl', # 'igl' or 'latlon'
        hemi_project='inflated', # Starting point for flattening (e.g., inflated, pial, sphere)
        morph=20, # Can expand or shrink the selection on the mesh (dilation/erosion)
        flat_name='v1_iglB',
        cut_box=True, 
    )
    # Morphing is useful to make sure the selection is not too small

In [62]:
pm.clear_flat(flat_name='occ')

before we go any further - lets check that it turned out how we would like

In [ ]:
# This is not the same as pycortex's quickshow
# pm.reload_flat('v1_igl')
x = np.hstack(
    [
        pm.mesh_info['sphere']['lh']['coords'][:,0],
        pm.mesh_info['sphere']['rh']['coords'][:,0]
    ]) 
pm.flat_mpl(
    data=x, #cmap='marco_pol', vmin=-np.pi, vmax=np.pi,
    data_mask=data_mask,
    flat_name='flat',
    roi_list='v1c', 
    # angle_rot=0  
    )

Above i use my own flat_mpl but to get this to work with pycortex you need to kill the kernel and rerun the notebook. up to this point

## Add the ROIs to pycortex SVG (Kill kernel and rerun first!)
To add ROIs we 

[1] Load the boolean array from freesurfer

[2] Find the vertices on the edge of ROIs

[3] Find the order for these vertices 

[4] Draw them onto our SVG file


Again I've written this so you can do it just using this line of code

In [23]:
roi_list = ['v1', ]  # roi names in you're freesurfer label folder
pm.add_rois_to_svg(roi_list)

In [ ]:

pm.quick_show()